# INBOX

See all incoming messages in this notebook. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to receive messages on your behalf. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import time 
import base64 
import getpass
import jwt
import json
import requests
from tools import print_with_timestamp, check_message_encryption

In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
data = input("Json Data:")
data_json= json.loads(data)
hmiSecret = data_json["hmiSecret"]
hmiId = data_json["hmi"]
username = data_json["username"]
password = data_json["pw"]
key = data_json["Key"]

In [ ]:
#hmiId = input('[S3I]: Please enter your HMI id:')
#hmiSecret = getpass.getpass('[S3I]: Please enter the secret:')
#print_with_timestamp("Client id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [ ]:
#print_with_timestamp("Workshop Inbox, please log in!")
#username = input('[S3I]: Please enter your username:')
#password = getpass.getpass('[S3I]: Please enter the password:')
#print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password',
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmiId,
                                             client_secret=hmiSecret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received " + parsed_username + " logged in")

In order to receive encrypted messages, you have to insert your personal key. The public key part of this key is available in the S³I directory. Participants, who want to send encrypted messages to you, encrypt their message with your public key. 

In [ ]:
#key = input('[S3I]: Please enter your key:')
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
personalKey = s3i.Key(key_str=key)
print_with_timestamp("The personal key of this hmi" + hmiId + "is set.")

In [ ]:
def receive(endpoint):
    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="rabbitmq.read:*/*/*")
    headers = {'Content-Type': 'application/pgp-encrypted',
               'Authorization': 'Bearer ' + access_token}
    response = requests.get(
        url="https://broker.s3i.vswf.dev/"+endpoint, headers=headers)
    return(response.text)

In [ ]:
def pgp_callback(pgp_message):
    message_str = pgp_message.replace("\\n", "\n").strip('"') 
    uMsg = s3i.UserMessage(msg_blob=message_str)

    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
    """Decrypt the message and verify the signature
    """
    if uMsg.pgpMsg.is_encrypted:
        uMsg.decryptAndVerify(personalKey, "", dir)
        #uMsg.decrypt(sec_key.key)
        uMsg.convertPgpToMsg()
    
    print_message(uMsg)

In [ ]:
def msg_callback(message):
    message_str = message.replace("'", '"')  # convert bytes to str
    uMsg = s3i.UserMessage(json_in=message_str)
    print_message(uMsg)

In [ ]:
def error_callback(message):
    print("The server returned:", message)
    print("==============================================================================")

In [ ]:
def print_message(uMsg):
    print_with_timestamp("Subject of the message: " + uMsg.msg["subject"])
    print_with_timestamp("Text of the message:  " + uMsg.msg["text"])
    print_with_timestamp("Sender of the message: " + dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))
    
    attachments_list = uMsg.msg["attachments"]
    """
    store the attachment file in specified path
    """
    for attachment in attachments_list:
        with open("received_data/"+attachment["filename"], 'wb') as file:
            decode = base64.b64decode(attachment["data"])
            file.write(decode)
            print("[S3I]: Attachment " + attachment["filename"]
               + " of the message is stored in received_data")
                        
    print("==============================================================================")

In [ ]:
def callback(message):
    switcher = {
        "pgp": pgp_callback,
        "error": error_callback,
        "msg": msg_callback
    }
    switcher.get(check_message_encryption(message), error_callback)(message)

    """print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: A new message has been received")
    message_str = message.replace("\\n", "\n").strip('"') 
    uMsg = s3i.UserMessage(msg_blob=message_str)

    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
    """
    #Decrypt the message and verify the signature
    """
    if uMsg.pgpMsg.is_encrypted:
        uMsg.decryptAndVerify(personalKey, "", dir)
        #uMsg.decrypt(sec_key.key)
        uMsg.convertPgpToMsg()

    print_with_timestamp("Subject of the message: " + uMsg.msg["subject"])
    print_with_timestamp("Text of the message:  " + uMsg.msg["text"])
    print_with_timestamp("Sender of the message: " + dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))
    
    attachments_list = uMsg.msg["attachments"]
    """
    #store the attachment file in specified path
    """
    for attachment in attachments_list:
        with open("received_data/"+attachment["filename"], 'wb') as file:
            decode = base64.b64decode(attachment["data"])
            file.write(decode)
            print("[S3I]: Attachment " + attachment["filename"]
               + " of the message is stored in received_data")
                        
    print("==============================================================================")"""

In [ ]:
endpoint = "s3ibs://"+hmiId
print("[S3I]: Start receiving messages as", endpoint)
while True:
    incomingMessage = receive(endpoint)
    if not(len(incomingMessage) == 0):
        callback(incomingMessage)
    else:
        time.sleep(2)